In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-sep-2021/train.csv
/kaggle/input/tabular-playground-series-sep-2021/test.csv
/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv


## Table of content
- A. Prepare data with Null values for the example
- B. Methods to handle the missing values
    - 1. Drop the data
    - 2. Fill the places with any of 5 Ms (Mean/Median/Mode/Max/Min)
    - 3. Fill the place with some constant value
    - 4. Predict the missing values
    - 5. Use models which support missing values

# A. Prepare data with Null values for the example


In [44]:
data_ = pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv",index_col = 0)

In [45]:
# keeping just 50 rows for my example.
def create_data():
    data = data_.iloc[0:50,30:50]
    return data
data = create_data()
print("Shape of the data------>{}".format(data.shape))
data.head()

Shape of the data------>(50, 20)


,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50
id,,,,,,,,,,,,,,,,,,,,
0,0.75050,18509.0,146820.0,-0.000276,1.090600e+16,1705.400,414.29,3.5392,1888.0,0.968930,18.3880,-0.001583,7.7059,5.9325,0.025693,4.5604,0.61122,10.7950,0.341930,0.235010
1,1.50330,238000.0,21440.0,-0.001344,3.079400e+16,229.100,844.82,1.4680,4726.5,0.915380,-1.5321,0.982600,7.1112,2.0797,0.042321,4.2523,0.41871,5.4499,0.012737,0.386470
2,1.13180,27940.0,862460.0,-0.002207,5.849100e+13,-897.840,NaN,1.3561,3063.4,0.086232,16.1060,0.001481,11.4760,5.3430,0.012162,4.1018,-0.88270,8.1228,-0.676690,0.337700
3,0.98941,301200.0,NaN,-0.000007,-9.299200e+13,-10.818,1020.30,2.9553,3342.5,-0.000372,17.0110,0.095268,5.7448,15.8830,0.037934,4.4860,-0.88909,8.4384,-1.189800,0.001391
4,0.97413,142620.0,231350.0,0.001257,1.012500e+16,51.508,293.76,1.3351,3042.1,0.006791,94.8890,0.917090,8.7369,NaN,0.020281,3.9115,0.65634,6.1410,-1.089600,0.247940


In [28]:
# find the missing values.
features = data.columns.tolist()[0:-1]


#find the missing values w.r.t. column
colum_missing = data.isnull().sum()
# find the missing values w.r.t. row(number of missing values in the particular row)
row_missing = data[features].isnull().sum(axis=1)

# add the missing values to row to the dataframe as a new value
data['no_of_missing_data'] = row_missing

In [29]:
colum_missing

f31    0
f32    1
f33    2
f34    1
f35    1
f36    2
f37    3
f38    0
f39    0
f40    0
f41    1
f42    1
f43    1
f44    2
f45    2
f46    0
f47    0
f48    1
f49    0
f50    0
dtype: int64

In [30]:
print(f"Total number of missing values in training dataset---->{data.shape[0]}")

# compare this to the whole data
no_of_missing_rows = (data['no_of_missing_data'] != 0).sum()
print("\n{0:{fill}{align}80}\n".format(" Data Summary " , fill = "=", align = "^"))
print(f"Total rows -----------------------> {data.shape[0]}\nNumber of rows has missing data---> {no_of_missing_rows}\n{'-'*50}\nNumber of rows has full data--------> {data.shape[0] - no_of_missing_rows}")


Total number of missing values in training dataset---->50

================================= Data Summary =================================

Total rows -----------------------> 50
Number of rows has missing data---> 17
--------------------------------------------------
Number of rows has full data--------> 33


# B. Methods to handle the missing values


## 1. Drop the data

In this method, we simply delete the rows or features/columns which has the Null value. We will delete a row if there are more missing values (say 70-75%) same goes for the columns. This is only preferred to use when we have enough samples in the dataset.  We can delete a feature/column when it has less feature importance over prediction. One has to make sure there is no add of bias, after we have removed the data.

Build-in functions:
- dropna() --> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html

Pros: 
1. Removing this unwanted data sometimes make our model more accurate.
2. Deleting a column with less importance is better, since there is no use of keeping this with full of null values and no use of speding time handling this.

Cons:
1. Loss of information.
2. Redure prediction accuracy --> when we have a big number of missing values.

In [40]:
# Example.
row_drop = create_data()
# Drop the rows with null value
row_drop.dropna( how = 'any' , inplace = True) # we can change how to any/all . if 'all' the row will be deleted when it has all values as null values.
row_drop.isnull().sum()

f31    0
f32    0
f33    0
f34    0
f35    0
f36    0
f37    0
f38    0
f39    0
f40    0
f41    0
f42    0
f43    0
f44    0
f45    0
f46    0
f47    0
f48    0
f49    0
f50    0
dtype: int64

In [41]:
# Drop the column --> here the f37  has '3' values. we can delete if we want to..
data_ = create_data()
data_ = data_.drop('f37', axis = 1)
data_.head()

,f31,f32,f33,f34,f35,f36,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50
id,,,,,,,,,,,,,,,,,,,
0,0.75050,18509.0,146820.0,-0.000276,1.090600e+16,1705.400,3.5392,1888.0,0.968930,18.3880,-0.001583,7.7059,5.9325,0.025693,4.5604,0.61122,10.7950,0.341930,0.235010
1,1.50330,238000.0,21440.0,-0.001344,3.079400e+16,229.100,1.4680,4726.5,0.915380,-1.5321,0.982600,7.1112,2.0797,0.042321,4.2523,0.41871,5.4499,0.012737,0.386470
2,1.13180,27940.0,862460.0,-0.002207,5.849100e+13,-897.840,1.3561,3063.4,0.086232,16.1060,0.001481,11.4760,5.3430,0.012162,4.1018,-0.88270,8.1228,-0.676690,0.337700
3,0.98941,301200.0,NaN,-0.000007,-9.299200e+13,-10.818,2.9553,3342.5,-0.000372,17.0110,0.095268,5.7448,15.8830,0.037934,4.4860,-0.88909,8.4384,-1.189800,0.001391
4,0.97413,142620.0,231350.0,0.001257,1.012500e+16,51.508,1.3351,3042.1,0.006791,94.8890,0.917090,8.7369,NaN,0.020281,3.9115,0.65634,6.1410,-1.089600,0.247940


## 2. Fill the places with any of 5 Ms (Mean/Median/Mode/Max/Min)

In this method, we can replace the null value with some approximations (Average(mean), Median, Mode, Min, and Max). This method can be used with the numerical columns. Even this is an approximate calculation for the null value, It is better than deleting the rows and columns. The Mean, Median, Mode are a statistical approach to handling the missing values. 

Build-in functions:
- fillna() --> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html ( with different methods)
- SimpleImputer--> https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html

Pros:
1. There is no loss of information.
2. Better approach with small dataset.

Cons:
1. Imputing the approximations add variance and bias.

### Example with SimpleImputer()
Chnage the strategy to ----> mean / median / most_frequent(mode)

In [48]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values = np.nan, strategy = "mean")

data_before = data.copy()
#print(data_before.isnull().sum())
data_after = pd.DataFrame(imputer.fit_transform(data_before))
#print(data_after.isnull().sum())

In [ ]:
# Example to replace with max in that column.
data_max = data.copy()
for f in data.columns.tolist():
    